In [1]:
from signal_processing import analyze_data, ecg_features, qrs_detection

import matplotlib
%matplotlib notebook
import neurokit2 as nk
import numpy as np
import pandas as pd
import os

### 1. QRS detection

In [4]:
patient = '001'           # Patient to analyze data
signal_to_present = 'i'  # Signal to show to the doctor
fs = 1000                 # Sample frequency
time_min_qrs = 0.06       # Minimum time for a qrs complex [1]
refractor_time = 0.2      # Minimum time between two qrs complexs [1]
th_grad = 0.075

In [5]:
wk_dir = os.getcwd()
path_name = f'{wk_dir}\\physionet.org\\files\\ptbdb\\1.0.0\\patient{patient}'

# Look for all the ecg files in one patient
files = [file for file in os.listdir(path_name) if 'xyz' in file]
files = np.unique(np.array([ file.split('.')[0] for file in files]))

# Create an object to save all waves from the same patient
waves = ecg_features.ecg_waves()

# Iterate over the ecg from one patient
for i, file in enumerate(files):
    if file == 'index':
        continue

    print(f'Patient number {patient}, ecg number {i}')

    # Extract data
    # signal_i to find the qrs complex, derivation 1 is the one used in this algorithm
    # signal_ii to find peaks, the one the doctor wants to see
    file = f'{path_name}\\{file}'
    df, col_names = analyze_data.upload_data(file)
    signal_ecg_i = analyze_data.select_derivation(df, col_names, der='i')
    signal_ecg_ii = analyze_data.select_derivation(df, col_names, der=signal_to_present)

    # Create object to find all features in an ecg
    wave_feat = ecg_features.ecg_wave(fs, signal_ecg_i, signal_ecg_ii)
    wave_feat.main_properties( fs=fs, t_qrs_min=time_min_qrs,  t_ref=refractor_time, th_grad=th_grad, plot = True)

    # Fill object with zeros in case the algorith does not work properly
    if type(wave_feat.periods) == bool:
        wave_feat.ecg_fix()

    # add ecg from the patient to the general object
    waves.add_wave(wave_feat)

print('yayy! We have it! ')

Patient number 001, ecg number 0
avg =  7.177029872218235e-06


<IPython.core.display.Javascript object>


 Heart rate = 82.8125 (beats/min) 
Heart rate periodicity = 0.7335 (mean seconds between beats) 
Heart rate variability = 0.00920271699010679 (std seconds between beats)
Patient number 001, ecg number 1
avg =  5.864781954041703e-06


<IPython.core.display.Javascript object>


 Heart rate = 86.45833333333333 (beats/min) 
Heart rate periodicity = 0.6956257668711657 (mean seconds between beats) 
Heart rate variability = 0.005270692076383867 (std seconds between beats)
Patient number 001, ecg number 2
avg =  -4.8425709291033166e-05


<IPython.core.display.Javascript object>


 Heart rate = 80.20833333333333 (beats/min) 
Heart rate periodicity = 0.7513774834437086 (mean seconds between beats) 
Heart rate variability = 0.00972383003184119 (std seconds between beats)
yayy! We have it! 


#### 2.1 Check the precision of the algorithm compare to neurokit2 library

In [28]:
# Select values to compare from one wave
for wave in waves.waves:
    ecg_signal_rdetection = wave.signal_1     # Select signal 2 because is the one we are showing to the doctor
    ecg_signal_peak_detection = wave.signal_2
    r_peaks_ind = wave.r_peaks_ind

    # apply the algorithm available in neurokit2 library to the same signal
    _, rpeaks = nk.ecg_peaks(ecg_signal_rdetection, sampling_rate=1000)
    rpeaks_ind = rpeaks['ECG_R_Peaks']

    # Calculate the difference in the number of r peaks
    print(f' Number of R peaks: Designed algorithm {len(r_peaks_ind)}, Neurokit2 {len(rpeaks_ind)}')

 Number of R peaks: Designed algorithm 170, Neurokit2 169
 Number of R peaks: Designed algorithm 144, Neurokit2 144
 Number of R peaks: Designed algorithm 151, Neurokit2 118


In [6]:
df_all_patients = pd.read_csv(f'{os.getcwd()}\\ecg_features_v12.csv')


In [7]:
# Error considering all patient
error_all_patients = df_all_patients['num_beats'] - df_all_patients['num beat neurokit2'].values

mean_error = np.mean(error_all_patients)
mean_abs_error = np.mean(abs(error_all_patients))
std_error = np.std(error_all_patients)
std_abs_error = np.std(abs(error_all_patients))

print(f'All patient: Mean error {mean_error} +- {std_error}')
print(f'All patient: Mean absolute error {mean_abs_error} +- {std_abs_error}')

# Error considering patients where the algorithm has worked
df_sub_patients = df_all_patients[df_all_patients["heart_rate"] > 0]
error_sub_patients = df_sub_patients['num_beats'] - df_sub_patients['num beat neurokit2']

All patient: Mean error -6.533962264150944 +- 31.730822128960163
All patient: Mean absolute error 10.356603773584906 +- 30.69655508564476


In [44]:
# Error considering patients where the algorithm has worked
df_sub_patients = df_all_patients[df_all_patients["heart_rate"] > 0]
error_sub_patients = df_sub_patients['num_beats'] - df_sub_patients['num beat neurokit2']

mean_error_sub = np.mean(error_sub_patients)
mean_abs_error_sub = np.mean(abs(error_sub_patients))
std_error_sub = np.std(error_sub_patients)
std_abs_error_sub = np.std(abs(error_sub_patients))
print(f'Sub patient: Mean error {mean_error_sub} +- {std_error_sub}')
print(f'Sub patient: Mean absolute error {mean_abs_error_sub} +- {std_abs_error_sub}')


Sub patient: Mean error -1.9568627450980391 +- 20.040109338885713
Sub patient: Mean absolute error 5.929411764705883 +- 19.242592607084276


In [41]:
wk_dir = os.getcwd()

df_save = pd.DataFrame()
patients = [patient[-3:] for patient in os.listdir(f'{wk_dir}\\physionet.org\\files\\ptbdb\\1.0.0')]

exception = ['OLS', 'tml', 'png', 'DME', 'ODS','txt']

it = 0
n = 1
for patient in patients:

    if patient in exception:
        continue
    if patient == '139':
        continue

    # Look for all the ecg files in one patient
    path_name = f'{wk_dir}\\physionet.org\\files\\ptbdb\\1.0.0\\patient{patient}'
    files = [file for file in os.listdir(path_name) if 'xyz' in file]
    files = np.unique(np.array([ file.split('.')[0] for file in files]))

    # Iterate over the ecg from one patient
    for i, file in enumerate(files):
        if file == 'index':
            continue

        if not (os.path.exists(f'{path_name}\\{file}.dat') and os.path.exists(f'{path_name}\\{file}.hea') and os.path.exists(f'{path_name}\\{file}.xyz')):
            continue

        print(f'Patient number {patient}, ecg number {i}')
        it += 1

        # Extract data
        # signal_i to find the qrs complex, derivation 1 is the one used in this algorithm
        # signal_ii to find peaks, the one the doctor wants to see
        file = f'{path_name}\\{file}'
        df, col_names = analyze_data.upload_data(file)
        signal_ecg_i = analyze_data.select_derivation(df, col_names, der='i')
        signal_ecg_ii = analyze_data.select_derivation(df, col_names, der='i')

        # Create object to find all features in an ecg
        wave_feat = ecg_features.ecg_wave(fs, signal_ecg_i, signal_ecg_ii)
        wave_feat.main_properties( fs=fs, t_qrs_min=time_min_qrs,  t_ref=refractor_time, th_grad=0.08, plot = False)

        if type(wave_feat.periods) == bool:
            wave_feat.ecg_fix()

        # apply the algorithm available in neurokit2 library to the same signal
        _, rpeaks = nk.ecg_peaks(signal_ecg_i, sampling_rate=1000)
        rpeaks_ind = rpeaks['ECG_R_Peaks']

        # Add values into the dataframe
        dic = {'patient':patient[-3:], 'heart_rate':wave_feat.heart_rate, 'periodicity':wave_feat.periodic_beat, 'variability':wave_feat.variability_beat, 'num_beats' :len(wave_feat.r_peaks_ind)+2,'num beat neurokit2': len(rpeaks_ind)}
        df_now = pd.DataFrame.from_records([dic])
        df_save = pd.concat([df_save,df_now])

        if (it == 30*n or patient == '294'):
            df_save.to_csv(f'{wk_dir}\\ecg_features_v12.csv')
            n +=1


Patient number 001, ecg number 0

 Heart rate = 82.8125 (beats/min) 
Heart rate periodicity = 0.7335 (mean seconds between beats) 
Heart rate variability = 0.00920271699010679 (std seconds between beats)
Patient number 001, ecg number 1

 Heart rate = 86.45833333333333 (beats/min) 
Heart rate periodicity = 0.6956257668711657 (mean seconds between beats) 
Heart rate variability = 0.005270692076383867 (std seconds between beats)
Patient number 001, ecg number 2

 Heart rate = 80.20833333333333 (beats/min) 
Heart rate periodicity = 0.7513774834437086 (mean seconds between beats) 
Heart rate variability = 0.00972383003184119 (std seconds between beats)
Patient number 002, ecg number 0

 Heart rate = 79.16666666666667 (beats/min) 
Heart rate periodicity = 0.760744966442953 (mean seconds between beats) 
Heart rate variability = 0.02103054804738505 (std seconds between beats)
Patient number 003, ecg number 0

 Heart rate = 72.39583333333333 (beats/min) 
Heart rate periodicity = 0.830088235294

C:\Users\escorihu\Miniconda3\envs\ecg_signal_processing_py_3.9\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\escorihu\Miniconda3\envs\ecg_signal_processing_py_3.9\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\escorihu\Miniconda3\envs\ecg_signal_processing_py_3.9\lib\site-packages\numpy\core\_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\escorihu\Miniconda3\envs\ecg_signal_processing_py_3.9\lib\site-packages\numpy\core\_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\escorihu\Miniconda3\envs\ecg_signal_processing_py_3.9\lib\site-packages\numpy\core\_methods.py:257: RuntimeWarning: invalid value en


 Heart rate = 1.4998500149985001 (beats/min) 
Heart rate periodicity = nan (mean seconds between beats) 
Heart rate variability = nan (std seconds between beats)
Patient number 219, ecg number 0

 Heart rate = 88.4911508849115 (beats/min) 
Heart rate periodicity = 0.6831091954022989 (mean seconds between beats) 
Heart rate variability = 0.4563340360352055 (std seconds between beats)
Patient number 220, ecg number 0

 Heart rate = 0 (beats/min) 
Heart rate periodicity = 0 (mean seconds between beats) 
Heart rate variability = 0 (std seconds between beats)
Patient number 221, ecg number 0

 Heart rate = 60.493950604939506 (beats/min) 
Heart rate periodicity = 1.0003983050847458 (mean seconds between beats) 
Heart rate variability = 0.022068514203751034 (std seconds between beats)
Patient number 222, ecg number 0

 Heart rate = 77.49225077492251 (beats/min) 
Heart rate periodicity = 0.7764276315789475 (mean seconds between beats) 
Heart rate variability = 0.06346772926100695 (std seconds

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\escorihu\\Desktop\\ecg_signal_processing\\physionet.org\\files\\ptbdb\\1.0.0\\patientRDS'